In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse(r"W:\Images\Emma Westlund\hs\16b8fbc7-3b7c-4928-8fba-74219c7fdaca\16b8fbc7-3b7c-4928-8fba.xml")

In [3]:
root = tree.getroot()

In [4]:
#get namespace
namespace = root.tag.split('}')[0].strip('{') if '}' in root.tag else ''

ns = {'ns':namespace}

In [5]:
# Find channel order

channels = tree.findall('.//ns:Sequence/ns:Record/ns:Channel', ns)

channel_order = []
for channel in channels:
    channel_order.append(channel.text)

print(channel_order)

['CellTracker Green', 'SiR700-DNA', 'Brightfield']


In [6]:
# Find position of images

# Step 1: Find Sublayouts- x and y coordinates of field for each well
# Step 2: Match well and sublayout ID
# Step 3: Save dict - key: well-id (ex, r01c01), value: [[x1,y1], [x2,y2] ...] for all fields

sublayouts = tree.findall('.//ns:Experiment/ns:Sublayouts/ns:Sublayout', ns)
wells = tree.findall('.//ns:Experiment/ns:MeasurementLayout/ns:Wells/ns:Well', ns)

well_layout = {}

for well in wells:
    col = well.find('ns:Col', ns).text
    row = well.find('ns:Row', ns).text

    well_name = "r"+row.zfill(2)+"c"+col.zfill(2)
    sublayout_id = int(well.find('ns:SublayoutID', ns).text)

    sublayout = sublayouts[sublayout_id-1]

    fields = sublayout.findall('ns:Field', ns)

    field_layout = []
    for field in fields:
        x = float(field.find('ns:X', ns).text)
        y = float(field.find('ns:Y', ns).text)
        field_layout.append([x,y])

    well_layout[well_name] = field_layout

print(well_layout)


{'r01c02': [[-0.00230468798828125, 0.001298109056640625]], 'r01c07': [[-0.0013060379638671876, 5.993741815185547e-05]], 'r01c09': [[-0.0008648657836914062, -0.001303065015625]], 'r01c10': [[-0.0001494866943359375, -0.00016331849462890624]], 'r02c08': [[-0.0008888718261718749, 0.0018982589589843749]], 'r02c09': [[-5.4506006240844734e-06, -0.0006722459238281249]], 'r03c03': [[-0.0005191791992187499, 0.0012122259999999998]], 'r03c04': [[-0.0009320825805664062, -0.0003673696115722656]], 'r03c08': [[-0.0011865460205078126, 0.000592870775390625]], 'r04c02': [[-0.0011193289794921876, 0.0001463591024169922]], 'r04c03': [[0.0015688050537109373, -0.0008282849252929686]], 'r04c04': [[0.0010844219970703123, -0.001255053052734375]], 'r04c05': [[0.0009067777709960937, 0.0010009730092773437]], 'r04c06': [[-0.0007208297119140624, 0.0007321056997070313]], 'r04c11': [[-0.0017669530029296875, 0.0007753165151367187]], 'r04c12': [[0.000537085205078125, 0.0013677259999999999]], 'r05c06': [[-1.50530049800872

In [20]:
# Get Pixel Size

# Get camera pixel size
camera_px = float(tree.find('.//ns:InstrumentDescription/ns:Cameras/ns:Camera/ns:PixelSizeX', ns).text)*10**6
# Get binning
binning = int(tree.find('.//ns:Experiment/ns:Exposures/ns:Exposure/ns:SimpleChannel/ns:CameraSetting/ns:BinningX', ns).text)
# Get objective magnification
M_objective = int(tree.find('.//ns:Experiment/ns:Exposures/ns:Exposure/ns:ObjectiveMagnification', ns).text)
M_factor = 1.087

pixel_size = (camera_px*binning)/(M_objective*M_factor) #um

In [24]:
# Generate TileConfiguration for Stitching
import os
top_text = ['# Define the number of dimensions we are working on', 'dim = 2', '# Define the image coordinates (in pixels)']

config_dir = r"./TileConfigurations"
if not os.path.exists(config_dir):
    os.makedirs(config_dir)

for well in well_layout.keys():
    cur_file = os.path.join(config_dir, f'TileConfiguration_{well}')

    fields = well_layout[well]

    with open(cur_file, 'w') as f:
        for i, field in enumerate(fields):
            image_name = well+'f'+str(i).zfill(2)
            x = field[0]/pixel_size
            y = field[1]/pixel_size

            f.write(image_name+f'; ; ({x}, {y})')
    f.close()


In [ ]:
# Get field overlap

overlap = int(tree.find('.//ns:Experiment/ns:Sublayouts/ns:Sublayout/ns:Definition/ns:OverlapX', ns).text)
print(overlap)

In [ ]:
# Get GUID and measurement name

guid = tree.find('ns:MeasurementID', ns).text
measurement_name = tree.find('ns:InitialPlateName', ns).text
print(guid, measurement_name)

In [ ]:
# Get number of planes

num_planes = int(tree.find('.//ns:Stack/ns:Planes', ns).text)
print(num_planes)

# Just moving through the children of the tree to get an idea of what's going on

In [ ]:
for child in root:
    print(child.tag)

In [ ]:
experiment = root[7]
for child in experiment:
    print(child.tag)

In [ ]:
sublayouts = experiment[6]
print(len(sublayouts))
for child in sublayouts:
    print(child.tag)